# ADK API Server + Streamlit — End-to-End Deep Dive

This notebook explains how to expose agents with the **Google ADK API Server**, how to **test** the endpoints, and how to build a minimal **Streamlit** client that talks to the API. It uses a simple **Q&A** agent that can optionally call an **MCP** tool (e.g., **Tavily** web search) for grounded answers.

## Table of Contents
1. Overview: Architecture & Flow
2. Preparing the Project
3. Starting the ADK API Server
4. Understanding the Endpoints
5. Testing the API (Swagger, curl, SSE vs JSON)
6. Response Events: What to Parse
7. Building a Small Streamlit App
8. Troubleshooting
9. Appendix: Sample Payloads
10. My Handles

## 1) Overview: Architecture & Flow

The pattern is **API-first**: you run the ADK API server and call it from any client (Streamlit, CLI, scripts).
```mermaid
flowchart LR
    subgraph Client
    UI[Streamlit App]:::u -->|HTTP| POST_RUN[/POST /run/]
    UI -->|HTTP| POST_SESSION[/POST /apps/.../sessions/.../]
    UI -->|HTTP| GET_APPS[/GET /list-apps/]
    end

    subgraph ADK_API[ADK API Server]
    RUN[(/run)]:::api --> AGENT[Agent: root_agent]:::agent
    GETS[(/list-apps, sessions, artifacts...)]:::api
    end

    AGENT -->|may call| MCP[MCP Tool Process e.g., Tavily]:::mcp

    classDef u fill:#f3f6ff,stroke:#b2c0ff;
    classDef api fill:#eefbf5,stroke:#b5e8cf;
    classDef agent fill:#fff7e6,stroke:#f3c57c;
    classDef mcp fill:#f7f0ff,stroke:#cab3ff;
```
**Key idea**: Your UI is just an HTTP client. The ADK API server loads your agent (a Python file exporting `root_agent`) and exposes endpoints for creating sessions and running turns.

## 2) Preparing the Project

### 2.1 Folder layout
Place agents under an `agents/` folder. Each agent package must export **`root_agent`**.

```
your-project/
├─ agents/
│  └─ simple/
│     ├─ __init__.py
│     └─ agent.py          # exports: root_agent
├─ .env                    # GOOGLE_API_KEY, TAVILY_API_KEY (if using Tavily), etc.
└─ ...
```

### 2.2 Minimal agent
Your `agents/simple/agent.py` should define a small LLM agent and export `root_agent`. If you want it to search when needed, attach an **MCP toolset** (e.g., Tavily).

```python
# agents/simple/agent.py (excerpt)
from google.adk.agents.llm_agent import Agent
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset, StdioServerParameters

root_agent = Agent(
    model="gemini-2.0-flash",
    name="simple",
    instruction="""
You are a helpful Q&A assistant. For factual/time-sensitive questions,
call an MCP web-search tool first, then synthesize a concise answer with brief citations.
""".strip(),
    tools=[
        MCPToolset(
            connection_params=StdioServerParameters(
                command="npx", args=["-y", "tavily-mcp"],
                env={"TAVILY_API_KEY": "<your key>"}
            )
        )
    ],
)
```

## 3) Starting the ADK API Server

Run the server **from your project root** (the directory that contains `agents/`).

```bash
adk api_server -v .
# Visit http://localhost:8000/docs for the Swagger UI
```

If your agents live in a non-default folder, set `ADK_AGENTS_DIR=agents`. Make sure `.env` is loaded in the same shell if your agent needs keys:

```bash
export ADK_AGENTS_DIR=agents
set -a
source .env
set +a
```

Common MCP launchers:
```bash
# Node-based
export MCP_TAVILY_COMMAND=npx
export MCP_TAVILY_ARGS="-y tavily-mcp"

# Or Python uvx
export MCP_TAVILY_COMMAND=uvx
export MCP_TAVILY_ARGS="tavily-mcp"
```

## 4) Understanding the Endpoints

| Method | Path | Purpose |
|---|---|---|
| `GET` | `/list-apps` | List discoverable apps/agents. |
| `POST` | `/apps/{app}/users/{user}/sessions/{session}` | Create a session (by id). |
| `GET` | `/apps/{app}/users/{user}/sessions` | List sessions for a user & app. |
| `GET` | `/apps/{app}/users/{user}/sessions/{session}` | Get one session (state, events). |
| `POST` | `/run` | Run one turn for an app+user+session with a user message. |
| `POST` | `/run_sse` | Stream events with server-sent events (if supported). |
| `GET` | `/apps/{app}/users/{user}/sessions/{session}/artifacts` | List artifact names. |
| `GET` | `/apps/{app}/users/{user}/sessions/{session}/artifacts/{name}` | Load an artifact. |

**/run** is the core endpoint. You send a user message in the request JSON. The response is either:
- a **list of event objects**, or
- a **dict** like `{ "events": [...] }`.

## 5) Testing the API (Swagger, curl, SSE vs JSON)

### 5.1 Swagger UI
Open **http://localhost:8000/docs**. You can create sessions and run turns directly from the browser.

### 5.2 curl (JSON response)
```bash
# Create a session
APP=simple
USER=demo_user
SID="session-$(date +%s)"
curl -s -X POST "http://localhost:8000/apps/$APP/users/$USER/sessions/$SID"   -H "Content-Type: application/json" -d '{}'

# Run one turn (non-streaming)
curl -s -X POST "http://localhost:8000/run"   -H "Content-Type: application/json"   -d '{
    "app_name": "'$APP'",
    "user_id": "'$USER'",
    "session_id": "'$SID'",
    "new_message": {"role":"user","parts":[{"text":"What is RAG?"}]}
  }' | jq .
```

### 5.3 curl (SSE response)
Some builds stream with **Server-Sent Events**. You’ll see lines like `data: { ... }`. To pretty-print as JSON:
```bash
curl -s -N -X POST "http://localhost:8000/run_sse"   -H "Content-Type: application/json"   -d '{
    "app_name": "'$APP'",
    "user_id": "'$USER'",
    "session_id": "'$SID'",
    "new_message": {"role":"user","parts":[{"text":"Who won Wimbledon 2024?"}]}
  }' | sed -n 's/^data: //p' | jq -s .
```

## 6) Response Events: What to Parse

A **response event** captures something that happened during the turn: model text, a tool call, a tool result, state/artifact deltas, etc.

```mermaid
sequenceDiagram
  participant C as Client (Streamlit)
  participant API as /run
  participant A as Agent
  participant MCP as Tavily MCP

  C->>API: POST /run (user message)
  API->>A: Invoke agent
  A->>MCP: (optional) tool call (search)
  MCP-->>A: tool result
  A-->>API: final events
  API-->>C: events (JSON or SSE)
```

### Typical shapes you will see

```json
{
  "author": "simple",
  "content": {
    "parts": [
      {"text": "Short answer here."}
    ],
    "role": "model"
  },
  "actions": {
    "stateDelta": {},
    "artifactDelta": {}
  }
}
```

If a **tool/function** was used, you’ll see function call/response entries. Queries are usually found under one of:

- `.functionCall.args.query`
- `.function_call.arguments.query`
- `.tool_request.arguments.query`

Your client can extract the **final text** by scanning `content.parts[].text` and keep the most recent non-empty string. For audit/debug, show the full events JSON.

## 7) Building a Small Streamlit App

The simplest app has three moves:

1. **Create a session** via `POST /apps/{app}/users/{user}/sessions/{session}`.  
2. **Run one turn** via `POST /run` with a user message.  
3. **Parse events** → show **final text** and any **search queries** from tool calls.

### Minimal app (key excerpts)
```python
import os, uuid, json, requests, streamlit as st

API_BASE = os.getenv("ADK_API_BASE", "http://localhost:8000")
APP_NAME = os.getenv("ADK_APP_SIMPLE", "simple")
USER_ID  = os.getenv("ADK_USER_ID", f"user-{uuid.uuid4()}")
session_id = None

def create_session():
    global session_id
    session_id = f"session-{uuid.uuid4()}"
    url = f"{API_BASE}/apps/{APP_NAME}/users/{USER_ID}/sessions/{session_id}"
    requests.post(url, json={}, headers={"Content-Type":"application/json"}).raise_for_status()

def run_turn(question: str):
    payload = {
        "app_name": APP_NAME, "user_id": USER_ID, "session_id": session_id,
        "new_message": {"role": "user", "parts": [{"text": question}]}
    }
    r = requests.post(f"{API_BASE}/run", json=payload, headers={"Content-Type":"application/json"})
    r.raise_for_status()
    return r.json()

def to_events(resp):
    return resp if isinstance(resp, list) else (resp.get("events") or [])

def final_text(events):
    text = ""
    for ev in events:
        for p in (ev.get("content") or {}).get("parts", []):
            if isinstance(p, dict) and isinstance(p.get("text"), str) and p["text"].strip():
                text = p["text"].strip()
    return text

def search_queries(events):
    out, seen = [], set()
    def walk(x):
        if isinstance(x, dict):
            for k in ("functionCall", "function_call", "tool_request"):
                if isinstance(x.get(k), dict):
                    args = x[k].get("args") or x[k].get("arguments") or {}
                    q = args.get("query")
                    if isinstance(q, str) and q.strip() and q not in seen:
                        seen.add(q); out.append(q.strip())
            for v in x.values(): walk(v)
        elif isinstance(x, list):
            for v in x: walk(v)
    walk(events)
    return out

# UI
st.button("Create Session", on_click=create_session)
q = st.text_input("Ask:", "What is retrieval augmented generation?")
if st.button("Run") and session_id:
    raw = run_turn(q)
    events = to_events(raw)
    st.write(final_text(events) or "_(No text)_")
    for s in search_queries(events):
        st.code(s)
```

You can also add two GETs for discovery:
```python
st.button("List Apps", on_click=lambda: st.json(requests.get(f"{API_BASE}/list-apps").json()))
st.button("List Sessions", on_click=lambda: st.json(requests.get(f"{API_BASE}/apps/{APP_NAME}/users/{USER_ID}/sessions").json()))
```

## 8) Troubleshooting

- **500 on `/run`**: ensure `agents/simple/agent.py` exports `root_agent`, `__init__.py` exists, you started server **from the project root**, and env keys are loaded in that shell.
- **Tool not called**: tighten instruction to require search first for factual queries, or ask a time-sensitive question to force tool usage.
- **SSE vs JSON**: some builds stream via `/run_sse`; for scripts, convert `data:` lines to JSON before parsing.
- **MCP launch**: set `MCP_TAVILY_COMMAND` to `npx` or `uvx` depending on your machine; ensure `TAVILY_API_KEY` is set.

## 9) Appendix: Sample Payloads

**Create Session**
```http
POST /apps/simple/users/demo_user/sessions/session-123
Content-Type: application/json

{}
```
Response:
```json
{"id":"session-123","appName":"simple","userId":"demo_user","events":[],"state":{}}
```

**Run (shape A: list)**
```json
[{"author":"simple","content":{"role":"model","parts":[{"text":"Short answer..."}]}}]
```
**Run (shape B: dict with events)**
```json
{"events":[{"author":"simple","content":{"role":"model","parts":[{"text":"Short answer..."}]}}]}
```

## 10) Connect with Me

Linkedin :- https://www.linkedin.com/in/mayank953/  
Youtube :- https://www.youtube.com/@tech.mayankagg  
Instagram :- https://www.instagram.com/tech.mayankagg/  
Substack :- https://aiwithmayank.substack.com  
Medium :- https://medium.com/@tech.mayankagg  
Udemy :- https://www.udemy.com/user/mayank-aggarwal-197/  
Github :- https://github.com/mayank953/